In [2]:
import cv2
import mediapipe as mp
import numpy as np
import face_recognition

# Set environment variable to force dlib to use CPU
import os

os.environ['DLIB_USE_CUDA'] = '0'

In [4]:
def fill_eyebrows_with_templates(image_path, left_eyebrow_template_path, right_eyebrow_template_path):
    # Load the image
    img = cv2.imread(image_path)

    # Initialize Mediapipe Face Detection
    mp_face_mesh = mp.solutions.face_mesh
    face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, min_detection_confidence=0.1)

    # Find face locations and facial landmarks using Mediapipe
    results = face_mesh.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

    if results.multi_face_landmarks:
        for landmarks in results.multi_face_landmarks:
            # Extract points for the left and right eyebrows using provided indices
            left_eyebrow_pts = np.array(
                [(int(landmarks.landmark[idx].x * img.shape[1]), int(landmarks.landmark[idx].y * img.shape[0])) for idx
                 in
                 [46, 53, 53, 52, 52, 65, 65, 55, 70, 63, 63, 105, 105, 66, 66, 107]], np.int32)

            right_eyebrow_pts = np.array(
                [(int(landmarks.landmark[idx].x * img.shape[1]), int(landmarks.landmark[idx].y * img.shape[0])) for idx
                 in
                 [276, 283, 283, 282, 282, 295, 295, 285, 300, 293, 293, 334, 334, 296, 296, 336]], np.int32)

            left_eyebrow_inpaint_pts = np.array(
                [(int(landmarks.landmark[idx].x * img.shape[1]), int(landmarks.landmark[idx].y * img.shape[0])) for idx
                 in
                 [70, 156, 156, 124, 124, 225, 225, 30, 30, 29, 29, 27, 27, 222, 222, 28, 28, 56, 56, 221, 9, 9, 70, 63,
                  63,
                  105, 105, 66, 66, 107, 107, 9]],
                np.int32)

            right_eyebrow_inpaint_pts = np.array(
                [(int(landmarks.landmark[idx].x * img.shape[1]), int(landmarks.landmark[idx].y * img.shape[0])) for idx
                 in
                 [300, 383, 383, 276, 276, 445, 445, 444, 444, 259, 259, 257, 257, 258, 258, 441, 441, 285, 285, 9, 300,
                  293,
                  293, 334, 334, 296,
                  296, 336, 336, 9]], np.int32)

            # Calculate the bounding boxes for the left and right eyebrows
            left_eyebrow_rect = cv2.boundingRect(left_eyebrow_pts)
            right_eyebrow_rect = cv2.boundingRect(right_eyebrow_pts)

            # Use in paint to remove eyebrows
            inpaint_mask = np.zeros_like(img, dtype=np.uint8)
            cv2.fillPoly(inpaint_mask, [left_eyebrow_inpaint_pts], (255, 255, 255))
            cv2.fillPoly(inpaint_mask, [right_eyebrow_inpaint_pts], (255, 255, 255))

            img = cv2.inpaint(img, inpaint_mask[:, :, 0], 100, cv2.INPAINT_TELEA)
            cv2.imshow('before smoothing', img)

            # Smoothing the left eyebrow region
            left_eyebrow_smoothed = cv2.GaussianBlur(
                img[left_eyebrow_rect[1]:left_eyebrow_rect[1] + left_eyebrow_rect[3],
                left_eyebrow_rect[0]:left_eyebrow_rect[0] + left_eyebrow_rect[2]], (5, 5), 0)

            # Smoothing the right eyebrow region
            right_eyebrow_smoothed = cv2.GaussianBlur(
                img[right_eyebrow_rect[1]:right_eyebrow_rect[1] + right_eyebrow_rect[3],
                right_eyebrow_rect[0]:right_eyebrow_rect[0] + right_eyebrow_rect[2]], (5, 5), 0)

            # Replace the inpainted eyebrow regions with the smoothed versions
            img[left_eyebrow_rect[1]:left_eyebrow_rect[1] + left_eyebrow_rect[3],
            left_eyebrow_rect[0]:left_eyebrow_rect[0] + left_eyebrow_rect[2]] = left_eyebrow_smoothed

            img[right_eyebrow_rect[1]:right_eyebrow_rect[1] + right_eyebrow_rect[3],
            right_eyebrow_rect[0]:right_eyebrow_rect[0] + right_eyebrow_rect[2]] = right_eyebrow_smoothed

            cv2.imshow('after smoothing', img)
            # Resize the eyebrow templates
            resized_left_eyebrow_template = cv2.resize(
                cv2.imread(left_eyebrow_template_path, cv2.IMREAD_UNCHANGED),
                (left_eyebrow_rect[2], left_eyebrow_rect[3]),
                interpolation=cv2.INTER_AREA,
            )

            resized_right_eyebrow_template = cv2.resize(
                cv2.imread(right_eyebrow_template_path, cv2.IMREAD_UNCHANGED),
                (right_eyebrow_rect[2], right_eyebrow_rect[3]),
                interpolation=cv2.INTER_AREA,
            )

            # Get the regions of interest (ROIs) for the eyebrows in the inpainted image
            left_roi = img[
                       left_eyebrow_rect[1]:left_eyebrow_rect[1] + left_eyebrow_rect[3],
                       left_eyebrow_rect[0]:left_eyebrow_rect[0] + left_eyebrow_rect[2],
                       ]
            right_roi = img[
                        right_eyebrow_rect[1]:right_eyebrow_rect[1] + right_eyebrow_rect[3],
                        right_eyebrow_rect[0]:right_eyebrow_rect[0] + right_eyebrow_rect[2],
                        ]

            # Putting templates on the roi area
            alpha_channel_left = resized_left_eyebrow_template[:, :, 3] / 255.0
            img_alpha_left = left_roi * (1 - alpha_channel_left[:, :, np.newaxis])
            resized_left_eyebrow = resized_left_eyebrow_template[:, :, :3] * (
                alpha_channel_left[:, :, np.newaxis]
            )
            roi_with_left_eyebrow = img_alpha_left + resized_left_eyebrow

            alpha_channel_right = resized_right_eyebrow_template[:, :, 3] / 255.0
            img_alpha_right = right_roi * (1 - alpha_channel_right[:, :, np.newaxis])
            resized_right_eyebrow = resized_right_eyebrow_template[:, :, :3] * (
                alpha_channel_right[:, :, np.newaxis]
            )
            roi_with_right_eyebrow = img_alpha_right + resized_right_eyebrow

            # Update the original inpainted image with the modified eyebrow regions
            img[
            left_eyebrow_rect[1]:left_eyebrow_rect[1] + left_eyebrow_rect[3],
            left_eyebrow_rect[0]:left_eyebrow_rect[0] + left_eyebrow_rect[2],
            ] = roi_with_left_eyebrow

            img[
            right_eyebrow_rect[1]:right_eyebrow_rect[1] + right_eyebrow_rect[3],
            right_eyebrow_rect[0]:right_eyebrow_rect[0] + right_eyebrow_rect[2],
            ] = roi_with_right_eyebrow

    # Display the result
    cv2.imshow('img', img)
    # cv2.imwrite('edited_pics/tom_cruise_with_eyebrows.png',img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


# Replace 'your_image_path.jpg' with the path to your image file
# Replace 'left_eyebrow_template.png' and 'right_eyebrow_template.png' with the paths to your eyebrow templates
fill_eyebrows_with_templates('samples_photos/tom_cruise.png', 'samples_photos/right1-eyebrow.png',
                             'samples_photos/left1-eyebrow.png')